# ML4CPS Project-2 | NB-6

In [ ]:
import os
import sys

basepath = os.path.abspath(os.path.join(".."))
if not basepath in sys.path:
    sys.path.append(basepath)

%load_ext autoreload
%autoreload 2

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
%matplotlib inline

In [ ]:
from utils.dataset import DatasetLoaderXL, DataProcessor, LocationProcessor

## Load dataset

In [ ]:
dsxl = DatasetLoaderXL(dataset_dir="../dataset/raw", seed=42, verbose=True)
dsxl.load_all_datasets()

In [ ]:
dsxl.list_suburbs()

In [ ]:
dsxl.list_categories()

In [ ]:
dsxl.list_subcategories("Hospital", console_print=False);

In [ ]:
dsxl.list_subcategories("Services", console_print=False);

In [ ]:
subcategories = [
    "Category 4 & 5 emergency department presentations",
    "Distance to nearest public hospital",
    "Distance to nearest public hospital with emergency department",
    "Distance to nearest public hospital with maternity services",
    "Obstetric type separations, 2012-13",
    "Presentations to emergency departments due to injury",
    "Presentations to emergency departments, 2012-13",
    "Public hospital separations, 2012-13",
    "Time to nearest public hospital with maternity services",
    "Travel time to nearest public hospital",
    "Travel time to nearest public hospital with emergency department",
    "Primary Schools",
]
df = dsxl.get_subcategories_across_all_suburbs(subcategories)

In [ ]:
df.replace('<5', np.nan, inplace=True)
df = df.apply(pd.to_numeric, errors='coerce')
df

In [ ]:
dp = DataProcessor(df)
dp.normalize()
pairs = dp.get_topk_abs_correlations(k=None, ascending=False)
pairs

In [ ]:
filtered_pairs = pairs[(pairs['Variable1'] == 'Primary Schools') | (pairs['Variable2'] == 'Primary Schools')]
pairs = filtered_pairs
pairs.reset_index(drop=True, inplace=True)
pairs

In [ ]:
pairs = pairs.apply(lambda row: row if row['Variable1'] == 'Primary Schools' else pd.Series({'Variable1': row['Variable2'], 'Variable2': row['Variable1'], 'Correlation': row['Correlation'], 'AbsCorrelation': row['AbsCorrelation']}), axis=1)
pairs.reset_index(drop=True, inplace=True)
pairs

In [ ]:
fig, axes = plt.subplots(nrows=pairs.shape[0], ncols=1, figsize=(10, 5*pairs.shape[0]))

for i, (index, row) in enumerate(pairs.iterrows()):
    ax = axes[i] if pairs.shape[0] > 1 else axes
    var1 = row['Variable1']
    var2 = row['Variable2']
    ax.scatter(dp.df[var1], dp.df[var2], alpha=0.5)
    ax.set_xlabel(var1)
    ax.set_ylabel(var2)
    ax.set_title(f"Correlation: {row['Correlation']:.2f}")
plt.tight_layout()
plt.show()